# Xây dựng hệ thống gợi ý phim (Content-Based) với Supabase

Notebook này hướng dẫn cách xây dựng một hệ thống gợi ý phim dựa trên nội dung, sử dụng dữ liệu từ Supabase. Khi người dùng chọn một bộ phim, hệ thống sẽ hiển thị danh sách các phim gợi ý dựa trên nội dung tương tự.

## 1. Kết nối tới Supabase và chuẩn bị dữ liệu

Trước tiên, bạn cần cài đặt thư viện `supabase` để kết nối tới Supabase từ Python.

In [23]:
from supabase import create_client, Client
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()
SUPABASE_URL = os.getenv('SUPABASE_URL')
SUPABASE_KEY = os.getenv('SUPABASE_KEY')

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Lấy dữ liệu từ bảng movies
response = supabase.table('movies').select('*').execute()
data = response.data
movies_df = pd.DataFrame(data)

# Xem trước dữ liệu
movies_df.head()

,id,title,overview,release_year,poster_url,genre,cast_ids,company_ids,keyword_ids,created_at,updated_at
0,1087192,How to Train Your Dragon,"On the rugged isle of Berk, where Vikings and ...",2025.0,https://image.tmdb.org/t/p/w500/41dfWUWtg1kUZc...,"[Fantasy, Family, Action]","[2803710, 2064124, 17276, 11109, 3792786, 1139...","[521, 2527]","[334, 5895, 9714, 12554, 14643, 192913, 245230...",2025-07-27T07:34:45.321093+00:00,2025-07-27T07:34:44.203+00:00
1,1311031,Demon Slayer: Kimetsu no Yaiba Infinity Castle,As the Demon Slayer Corps members and Hashira ...,2025.0,https://image.tmdb.org/t/p/w500/aFRDH3P7TX61FV...,"[Animation, Action, Fantasy, Thriller]","[1256603, 1563442, 233590, 119145, 90571, 1452...","[5887, 2883, 2918]","[6152, 9663, 13141, 15001, 207826, 210024, 227...",2025-07-27T07:34:45.321093+00:00,2025-07-27T07:34:44.203+00:00
2,552524,Lilo & Stitch,The wildly funny and touching story of a lonel...,2025.0,https://image.tmdb.org/t/p/w500/c32TsWLES7kL1u...,"[Family, Science Fiction, Comedy, Adventure]","[3988423, 3025125, 66193, 58225, 141034, 24047...","[2, 118854]","[1668, 6733, 10041, 9951, 14549, 14729, 18035,...",2025-07-27T07:34:45.321093+00:00,2025-07-27T07:34:44.203+00:00
3,1071585,M3GAN 2.0,After the underlying tech for M3GAN is stolen ...,2025.0,https://image.tmdb.org/t/p/w500/oekamLQrwlJjRN...,"[Action, Science Fiction, Thriller]","[2043430, 1255540, 2131391, 1243496, 3444018, ...","[3172, 76907, 89115]","[803, 310, 1721, 1373, 1829, 3561, 9663, 9707,...",2025-07-27T07:34:45.321093+00:00,2025-07-27T07:34:44.203+00:00
4,617126,The Fantastic 4: First Steps,Against the vibrant backdrop of a 1960s-inspir...,2025.0,https://image.tmdb.org/t/p/w500/x26MtUlwtWD26d...,"[Science Fiction, Adventure]","[1253360, 556356, 21042, 1597365, 202032, 9369...","[420, 176762]","[9715, 9717, 18035, 175629, 180547, 208992, 21...",2025-07-27T07:34:45.321093+00:00,2025-07-27T07:34:44.203+00:00


In [24]:
# Tiền xử lý dữ liệu: loại bỏ các giá trị thiếu và chuẩn hóa tên cột
movies_df = movies_df.dropna().reset_index(drop=True)
movies_df.columns = movies_df.columns.str.strip().str.lower().str.replace(' ', '_')

# Xem lại dữ liệu sau khi tiền xử lý
movies_df.head()

,id,title,overview,release_year,poster_url,genre,cast_ids,company_ids,keyword_ids,created_at,updated_at
0,1087192,How to Train Your Dragon,"On the rugged isle of Berk, where Vikings and ...",2025.0,https://image.tmdb.org/t/p/w500/41dfWUWtg1kUZc...,"[Fantasy, Family, Action]","[2803710, 2064124, 17276, 11109, 3792786, 1139...","[521, 2527]","[334, 5895, 9714, 12554, 14643, 192913, 245230...",2025-07-27T07:34:45.321093+00:00,2025-07-27T07:34:44.203+00:00
1,1311031,Demon Slayer: Kimetsu no Yaiba Infinity Castle,As the Demon Slayer Corps members and Hashira ...,2025.0,https://image.tmdb.org/t/p/w500/aFRDH3P7TX61FV...,"[Animation, Action, Fantasy, Thriller]","[1256603, 1563442, 233590, 119145, 90571, 1452...","[5887, 2883, 2918]","[6152, 9663, 13141, 15001, 207826, 210024, 227...",2025-07-27T07:34:45.321093+00:00,2025-07-27T07:34:44.203+00:00
2,552524,Lilo & Stitch,The wildly funny and touching story of a lonel...,2025.0,https://image.tmdb.org/t/p/w500/c32TsWLES7kL1u...,"[Family, Science Fiction, Comedy, Adventure]","[3988423, 3025125, 66193, 58225, 141034, 24047...","[2, 118854]","[1668, 6733, 10041, 9951, 14549, 14729, 18035,...",2025-07-27T07:34:45.321093+00:00,2025-07-27T07:34:44.203+00:00
3,1071585,M3GAN 2.0,After the underlying tech for M3GAN is stolen ...,2025.0,https://image.tmdb.org/t/p/w500/oekamLQrwlJjRN...,"[Action, Science Fiction, Thriller]","[2043430, 1255540, 2131391, 1243496, 3444018, ...","[3172, 76907, 89115]","[803, 310, 1721, 1373, 1829, 3561, 9663, 9707,...",2025-07-27T07:34:45.321093+00:00,2025-07-27T07:34:44.203+00:00
4,617126,The Fantastic 4: First Steps,Against the vibrant backdrop of a 1960s-inspir...,2025.0,https://image.tmdb.org/t/p/w500/x26MtUlwtWD26d...,"[Science Fiction, Adventure]","[1253360, 556356, 21042, 1597365, 202032, 9369...","[420, 176762]","[9715, 9717, 18035, 175629, 180547, 208992, 21...",2025-07-27T07:34:45.321093+00:00,2025-07-27T07:34:44.203+00:00


### Tiền xử lý văn bản cho thuộc tính overview

Để tăng hiệu quả cho hệ thống gợi ý, nên thực hiện các bước tiền xử lý văn bản cho trường overview như: chuyển về chữ thường, loại bỏ ký tự đặc biệt, loại bỏ stopwords, ...

In [25]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk_stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Chuyển về chữ thường
    text = text.lower()
    # Loại bỏ ký tự đặc biệt và số
    text = re.sub(r'[^a-z\s]', '', text)
    # Tách từ
    words = text.split()
    # Loại bỏ stopwords và đưa về dạng nguyên mẫu
    words = [lemmatizer.lemmatize(w) for w in words if w not in nltk_stopwords]
    return ' '.join(words)

# Áp dụng tiền xử lý cho overview
movies_df['overview'] = movies_df['overview'].astype(str).apply(preprocess_text)

# Xem kết quả sau tiền xử lý
movies_df[['title', 'overview']].head()

,title,overview
0,How to Train Your Dragon,rugged isle berk viking dragon bitter enemy ge...
1,Demon Slayer: Kimetsu no Yaiba Infinity Castle,demon slayer corp member hashira engaged group...
2,Lilo & Stitch,wildly funny touching story lonely hawaiian gi...
3,M3GAN 2.0,underlying tech mgan stolen misused powerful d...
4,The Fantastic 4: First Steps,vibrant backdrop sinspired retrofuturistic wor...


In [26]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            993 non-null    int64  
 1   title         993 non-null    object 
 2   overview      993 non-null    object 
 3   release_year  993 non-null    float64
 4   poster_url    993 non-null    object 
 5   genre         993 non-null    object 
 6   cast_ids      993 non-null    object 
 7   company_ids   993 non-null    object 
 8   keyword_ids   993 non-null    object 
 9   created_at    993 non-null    object 
 10  updated_at    993 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 85.5+ KB


In [27]:
# Tạo cột đặc trưng tổng hợp cho mỗi phim từ các trường quan trọng
def combine_features(row):
    return ' '.join([
        str(row['title']),
        str(row['overview']),
        str(row['genre']),
        str(row['cast_ids']),
        str(row['company_ids']),
        str(row['keyword_ids'])
    ])

movies_df['features'] = movies_df.apply(combine_features, axis=1)

# Xem thử cột features mới
movies_df[['title', 'features']].head()

,title,features
0,How to Train Your Dragon,How to Train Your Dragon rugged isle berk viki...
1,Demon Slayer: Kimetsu no Yaiba Infinity Castle,Demon Slayer: Kimetsu no Yaiba Infinity Castle...
2,Lilo & Stitch,Lilo & Stitch wildly funny touching story lone...
3,M3GAN 2.0,M3GAN 2.0 underlying tech mgan stolen misused ...
4,The Fantastic 4: First Steps,The Fantastic 4: First Steps vibrant backdrop ...


### 3. Vector hóa đặc trưng (feature vectorization)

Ở bước này, ta sẽ sử dụng kỹ thuật TF-IDF để chuyển cột `features` thành các vector số, phục vụ cho việc tính toán độ tương đồng giữa các phim.

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Khởi tạo vectorizer và vector hóa cột features
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['features'])

# Kích thước ma trận đặc trưng
tfidf_matrix.shape

(993, 20545)

### 4. Tính toán độ tương đồng giữa các phim

Sử dụng cosine similarity để tính toán mức độ tương đồng giữa các vector đặc trưng của các phim.

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

# Tính toán ma trận độ tương đồng giữa các phim
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Xem kích thước ma trận và một phần giá trị
print(cosine_sim.shape)
cosine_sim[:5, :5]

(993, 993)


array([[1.        , 0.0029293 , 0.0190302 , 0.00156802, 0.01929774],
       [0.0029293 , 1.        , 0.        , 0.00443358, 0.0173045 ],
       [0.0190302 , 0.        , 1.        , 0.0065686 , 0.03876427],
       [0.00156802, 0.00443358, 0.0065686 , 1.        , 0.00666094],
       [0.01929774, 0.0173045 , 0.03876427, 0.00666094, 1.        ]])

### 5. Xây dựng hàm gợi ý phim dựa trên độ tương đồng

Hàm dưới đây sẽ nhận vào tên phim và trả về danh sách các phim tương tự nhất dựa trên ma trận độ tương đồng đã tính toán.

In [30]:
def recommend_movies(title, movies_df, cosine_sim, top_n=10):
    # Lấy chỉ số của phim theo tên
    idx = movies_df[movies_df['title'].str.lower() == title.lower()].index
    if len(idx) == 0:
        print(f'Không tìm thấy phim "{title}" trong dữ liệu.')
        return []
    idx = idx[0]
    # Lấy danh sách các phim và điểm tương đồng
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sắp xếp theo điểm tương đồng giảm dần, bỏ qua chính nó (idx)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    # Lấy chỉ số các phim tương tự
    movie_indices = [i[0] for i in sim_scores]
    # Trả về DataFrame các phim gợi ý với các cột id, title, poster_url
    return movies_df.iloc[movie_indices][['id', 'title', 'poster_url']]

# Ví dụ sử dụng:
recommend_movies('The Matrix', movies_df, cosine_sim, top_n=5)

,id,title,poster_url
877,604,The Matrix Reloaded,https://image.tmdb.org/t/p/w500/9TGHDvWrqKBzwD...
991,605,The Matrix Revolutions,https://image.tmdb.org/t/p/w500/t1wm4PgOQ8e4z1...
529,218,The Terminator,https://image.tmdb.org/t/p/w500/hzXSE66v6KthZ8...
155,27205,Inception,https://image.tmdb.org/t/p/w500/oYuLEt3zVCKq57...
967,62,2001: A Space Odyssey,https://image.tmdb.org/t/p/w500/ve72VxNqjGM69U...


In [31]:
# Lưu movies_df thành file CSV
movies_df.to_csv('movies_df.csv', index=False)

# Lưu tfidf_matrix thành file pickle
import pickle
with open('tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)